In [ ]:
from PIL import Image
import pytesseract
from pytesseract import Output

print(pytesseract.image_to_data(Image.open('files/textOriginal.png'),  output_type=Output.DICT))

In [ ]:
print(pytesseract.image_to_string(Image.open('files/textOriginal.png'), output_type=Output.BYTES))

In [ ]:
#провекра Numpy
import numpy as np

numbers = [100, 102, 98, 97, 103]
print(np.std(numbers))
print(np.mean(numbers))

In [ ]:
from PIL import Image
import subprocess

def cleanFile(filePath, newFilePath):
    image = Image.open(filePath)

    #установка порога для изображения
    image = image.point(lambda x: 0 if x<135 else 255)
    image.save(newFilePath)

    #вызов tesseract для OCR изображения
    subprocess.call(['tesseract', newFilePath, 'output'])
    
    #запись результата OCR в текстовый файл
    outputFile = open('output.txt', 'r')
    print(outputFile.read())
    outputFile.close()

cleanFile('files/textOriginal.png', 'files/text_2.png')

In [ ]:
from PIL import Image
import pytesseract

def cleanFile(filePath, newFilePath):
    image = Image.open(filePath)

      #установка порога для изображения (для выделения текста)
    image = image.point(lambda x: 0 if x<143 else 255)
    image.save(newFilePath)
    return image

image = cleanFile('files/textBad.png', 'files/textCleaned.png')

#вызов tesseract для OCR изображения
print(pytesseract.image_to_string(image))

In [ ]:
import pytesseract
from pytesseract import Output
from PIL import Image
import numpy as np

def cleanFile(filePath, threshold):
    image = Image.open(filePath)
    #установка порога для изображения
    image = image.point(lambda x: 0 if x<threshold else 255)
    return image

def getConfidence(image):
    data = pytesseract.image_to_data(image, output_type=Output.DICT)
    text = data['text']
    confidences = []
    numChars = []
    print(type(data['conf'][0]))
    for i in range(len(text)):
        if  int(data['conf'][i])>-1:
            confidences.append(int(data['conf'][i]))
            numChars.append(len(text[i]))
            
    return np.average(confidences, weights=numChars), sum(numChars)
    
filePath = 'files/textBad.png'

start = 80
step = 5
end = 200

for threshold in range(start, end, step):
    image = cleanFile(filePath, threshold)
    scores = getConfidence(image)
    print('threshold: {}, confidence: {}, numChars {}'.format(str(threshold), str(scores[0]), str(scores[1])))


Бот щелкает кнопку предпросмотра

In [ ]:
import time
from urllib.request import urlretrieve
from PIL import Image
import pytesseract
from selenium import webdriver
from PIL import Image

# создаем Selenium driver
driver = webdriver.Chrome(executable_path='drivers/chromedriver')

driver.get(
    'https://www.amazon.com/Death-Ivan-Ilyich-Nikolayevich-Tolstoy/dp/1427027277')
time.sleep(2)

# Щелкаем на кнопку предпросмотра с элементом 'imgBlkFront'
driver.find_element_by_id('imgBlkFront').click()
imageList = []

# ждем, пока страница загрузится
time.sleep(5)

while 'pointer' in driver.find_element_by_id('sitbReaderRightPageTurner').get_attribute('style'):
    # щелаекм пока доступна кнопка вправо для загрузки следующей страницы предпросмотра
    driver.find_element_by_id('sitbReaderRightPageTurner').click()
    time.sleep(2)
    # получаем все загруженные страницы (может сразу загрузиться несколько страниц,
    # но дубликаты игнорируютсяt)
    pages = driver.find_elements_by_xpath(
        '//div[@class=\'pageImage\']/div/img')
    if not len(pages):
        print('No pages found')
    for page in pages:
        image = page.get_attribute('src')
        print('Found image: {}'.format(image))
        if image not in imageList:
            urlretrieve(image, 'page.jpg')
            imageList.append(image)
            print(pytesseract.image_to_string(Image.open('page.jpg')))

driver.quit()

Бот добавляет комментарии, заполняет форму, читает CAPTCHA

In [ ]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess
import requests
from PIL import Image
from PIL import ImageOps

def cleanImage(imagePath):
    image = Image.open(imagePath)
    image = image.point(lambda x: 0 if x<143 else 255)
    borderImage = ImageOps.expand(image,border=20,fill='white')
    borderImage.save(imagePath)

html = urlopen('http://www.pythonscraping.com/humans-only')
bs = BeautifulSoup(html, 'html.parser')
#Данные для формы
imageLocation = bs.find('img', {'title': 'Image CAPTCHA'})['src']
formBuildId = bs.find('input', {'name':'form_build_id'})['value']
captchaSid = bs.find('input', {'name':'captcha_sid'})['value']
captchaToken = bs.find('input', {'name':'captcha_token'})['value']

captchaUrl = 'http://pythonscraping.com'+imageLocation
urlretrieve(captchaUrl, 'captcha.jpg')
cleanImage('captcha.jpg')
p = subprocess.Popen(['tesseract', 'captcha.jpg', 'captcha'], stdout=
    subprocess.PIPE,stderr=subprocess.PIPE)
p.wait()
f = open('captcha.txt', 'r')

#Удаляем символы пробела
captchaResponse = f.read().replace(' ', '').replace('\n', '')
print('Попытка распознавания Captcha: '+captchaResponse)

if len(captchaResponse) == 5:
    params = {'captcha_token':captchaToken, 'captcha_sid':captchaSid,   
              'form_id':'comment_node_page_form', 'form_build_id': formBuildId, 
              'captcha_response':captchaResponse, 'name':'Monty Python', 
              'subject': 'I come to seek the Grail', 
              'comment_body[und][0][value]': 
               '...and I am definitely not a bot'}
    r = requests.post('http://www.pythonscraping.com/comment/reply/10', 
                          data=params)
    responseObj = BeautifulSoup(r.text, 'html.parser')
    if responseObj.find('div', {'class':'messages'}) is not None:
        print(responseObj.find('div', {'class':'messages'}).get_text())
else:
    print('Возникли проблемы при распознавании CAPTCHA (неверное число символов)!')

# Распознавание русского текста

In [ ]:
rustext = pytesseract.image_to_string(Image.open('files/rustext.png'))
print(rustext)

In [ ]:
# Указываем в параметрах русския язык
rustext = pytesseract.image_to_string(Image.open('files/rustext.png'), lang='rus')
print(rustext)


In [ ]:
#Пример изображения с текстом. Текст необходимо выделить.
import matplotlib.pyplot as plt
img=Image.open('files/examen-student.png')
plt.imshow(img)
img.show()


In [ ]:
#print( pytesseract.image_to_string(img, lang='rus')

In [ ]:
#Задание:  получить весь текст. Модифицировать функцию для очистки изображения!
from PIL import Image, ImageEnhance
import pytesseract

def cleanFile(filePath, newFilePath):
    image = Image.open(filePath)
    image=ImageEnhance.Brightness(image).enhance(10.0)
    image=ImageEnhance.Contrast(image).enhance(10.0)
    image=image.convert('1')
      #установка порога для изображения (для выделения текста)
    #image = image.point(lambda x: 0 if x< else 255)
    image.save(newFilePath)
    return image

image = cleanFile('files/examen-student.jpg', 'files/examen-student-Cleaned.jpg')
#image.show()

#print( pytesseract.image_to_string(Image.open('files/examen-student-Cleaned.png'), lang='rus')

In [ ]:
#Задание: получить весь текст.
#Пример изображения с текстом. Текст необходимо выделить.
import matplotlib.pyplot as plt
img=Image.open('files/examen-prepod2.jpg')
plt.imshow(img)
img.show()